# Import dataset

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("michiard/dcase-aml")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dcase-aml


In [4]:
import os
import pandas as pd

notebook_executed_in="kaggle" # or colab

if notebook_executed_in == "kaggle":
    train_dir = '/kaggle/input/dcase-aml/dev_data/dev_data/slider/train'
elif notebook_executed_in == "colab":
    train_dir = 'dcase-aml/dev_data/_dev_data/slider/train'  # TODO: verify path in Colab
else:
    raise ValueError(f"Unsupported notebook: {notebook_executed_in}")

    
# List all files in the train directory
file_names = os.listdir(train_dir)

# Optional: filter to include only certain file types (e.g., .wav, .csv, etc.)
# file_names = [f for f in file_names if f.endswith('.wav')]

# Create a DataFrame
df = pd.DataFrame(file_names, columns=['filename'])

# Display the DataFrame
df.head()


,filename
0,normal_id_02_00000546.wav
1,normal_id_02_00000764.wav
2,normal_id_00_00000530.wav
3,normal_id_02_00000820.wav
4,normal_id_02_00000102.wav


## Preprocessing : build melspectrograms from audio files

- Input : audio files in data/data
- Output : melspectrograms of size

In [3]:
%matplotlib inline
import librosa
import librosa.display
import IPython.display as ipd
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
from tqdm import tqdm

## Choose the paths

In [24]:
# Enter the path to audio data
PATH_DATASET = "/kaggle/working/"
# Enter the path where spectrograms will be saved
PATH_FEATURES = PATH_DATASET+'Features/'
PATH_MELSPEC = PATH_FEATURES + 'melspec_313_128/'


## Function to build the dataframe of file names with their features for each machine type

In [37]:
#Get list of files in train and test directory
def get_df_files(machine_str):

    path_train_folder = '/kaggle/input/dcase-aml/dev_data/dev_data/slider/train'
    path_test_folder = '/kaggle/input/dcase-aml/dev_data/dev_data/slider/test'
    
    if not os.path.exists(path_train_folder):
        os.makedirs(path_train_folder)
        
    if not os.path.exists(path_test_folder):
        os.makedirs(path_test_folder)


    train_files = [f for f in os.listdir(path_train_folder)]
    test_files = [f for f in os.listdir(path_test_folder)]

    #Get list of dictionnary for creating DataFrame
    list_dict_file = []

    #Loop through filenames
    for filename in train_files:

        #Get filename as list of string
        splitted = filename.replace('.wav', '').split('_')
        #Append dictionnary to list
        list_dict_file.append({
            'filepath' : path_train_folder + "/" + filename,
            'filename': filename,
            'status': splitted[0],            # normal / anomaly
            'machine_id': splitted[1] + '_' + splitted[2],  # id_00
            'file_index': splitted[3]         # progressivo
        })

    #Loop through filenames
    for filename in test_files:

         #Get filename as list of string
        splitted = filename.replace('.wav', '').split('_')
        #Append dictionnary to list
        list_dict_file.append({
            'filepath' : path_test_folder + "/" + filename,
            'filename': filename,
            'status': splitted[0],            # normal / anomaly
            'machine_id': splitted[1] + '_' + splitted[2],  # id_00
            'file_index': splitted[3]         # progressivo
        })

    #Get file dataframe
    df_files = pd.DataFrame(list_dict_file)
    
    return df_files

## Functions to compute and save the spectograms

In [38]:
def build_melspectrogram(audio_name, spectro_params):
    #Get audio array
    audio_array, sr = librosa.load(audio_name, sr=None)

    #Get mel spectogram
    melspectrogram = librosa.feature.melspectrogram(
        y = audio_array, 
        n_fft = spectro_params['n_fft'], 
        hop_length = spectro_params['hop_length'], 
        n_mels = spectro_params['n_mels']
    )
    # Express in decibel units
    melspectrogram = librosa.power_to_db(melspectrogram, ref=1e-12)   #  ref=np.max 
    # Flatten the spectrogram
    #print("Initial Spectro shape = ", melspectrogram.shape)
    melspectrogram = melspectrogram.reshape(1, -1)
    #print("Final Spectro shape = ", melspectrogram.shape)
    
    return melspectrogram


def compute_and_save_spectros(machine_str, spectro_params):
    # Check or create folders
    if not os.path.exists(PATH_FEATURES):
        os.makedirs(PATH_FEATURES)

    if not os.path.exists(PATH_MELSPEC):
        os.makedirs(PATH_MELSPEC)

    path_machine_folder = PATH_MELSPEC + machine_str + '/'
    if not os.path.exists(path_machine_folder):
        os.makedirs(path_machine_folder)
        
    # Get the dataframe of files for this machine
    df = get_df_files(machine_str)
    
    # Loop through the dataframe
    for idx, row in df.iterrows():
        # Get audio array
        audio_path = row['filepath']
        # Build the spectrogram
        melspectrogram = build_melspectrogram(audio_path, spectro_params)   
        # Save 
        filename = path_machine_folder + row['filename'].split('.wav')[0]
        np.save(filename, melspectrogram)

## Run it for all machine types

In [39]:
spectro_params = dict(
    n_fft = 1024,     # length of the FFT window
    hop_length = 512,    # number of samples between successive frames
    n_mels = 128       # number of Mel bands to generate
)

machines_list = ['slider']

for machine in tqdm(machines_list):
    compute_and_save_spectros(machine, spectro_params)


100%|██████████| 1/1 [01:43<00:00, 103.17s/it]


### Test

In [45]:
audio_name = '/kaggle/input/dcase-aml/dev_data/dev_data/slider/train/normal_id_00_00000000.wav'

spectro_params = dict(
    n_fft = 1024, 
    hop_length = 512, 
    n_mels=128
)

spectro = build_melspectrogram(audio_name, spectro_params)

## Plot one spectrogram for test

In [46]:
def plot_spectro(filepath):
    # Load the original melspectogram array
    spectro_array = np.load(filepath).reshape(128, 313)
    # Plot the spectrogram
    librosa.display.specshow(spectro_array, sr = 16000, x_axis = 'time', y_axis = 'mel', cmap = 'magma')


In [47]:
path = '../data/Features/melspec_313_128/slider/section_00_source_test_anomaly_0000_vel_1100.npy'
plot_spectro(path)

FileNotFoundError: [Errno 2] No such file or directory: '../data/Features/melspec_313_128/slider/section_00_source_test_anomaly_0000_vel_1100.npy'